In [63]:
from collections import namedtuple

Utilities = namedtuple('Utilities', ['tid', 'pu', 'nu', 'ru'])

class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu
    
    @twu.setter
    def twu(self, value: int) -> None:
        if value < 0:
            raise ValueError("TWU value cannot be negative!")
        self._twu = value

    def __repr__(self):
        return f"({self.item},{self.twu})"
        # return f"{self.item}"

class Transaction:
    def __init__(self, id: int, items_quantities: dict):
        if any(q <= 0 for q in items_quantities.values()):
            raise ValueError(f"Quantities in trans{id} must be positive integers.")
        self.id = id
        self.items_quantities = items_quantities

    def __repr__(self):
        return f"(tid = {self.id}, frequencies = {self.items_quantities})"

class PNUList:
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        self.items = items
        self.utility_values = utility_values

    def add_item(self, item):
        self.items.add(item)

    def __repr__(self):
        return f"PNU-List(items={self.items}, " f"utility_values={self.utility_values})"


def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0
    items: set[Item] = trans.items_quantities.keys()
    for item in items:
        if item.utility > 0:
            pu += trans.items_quantities.get(item) * item.utility
    return pu


def calculate_transaction_weight_utility(items: set[Item], database: list[Transaction]):
    twu = 0
    for trans in database:
        trans_items = trans.items_quantities.keys()
        if items.issubset(trans_items):
            twu += calculate_positive_utility_of_transaction(trans)
    return twu

In [ ]:
a = Item("A", -5)
b = Item("B", 2)
c = Item("C", 1)
d = Item("D", 2)
e = Item("E", 3)
f = Item("F", 1)
g = Item("G", 1)

trans1 = Transaction(1, dict(zip([a, c, d], [1, 1, 1])))
trans2 = Transaction(2, dict(zip([a, c, e, g], [2, 6, 2, 5])))
trans3 = Transaction(3, dict(zip([a, b, c, d, e, f], [1, 2, 1, 6, 1, 5])))
trans4 = Transaction(4, dict(zip([b, c, d, e], [4, 3, 3, 1])))
trans5 = Transaction(5, dict(zip([b, c, e, g], [2, 2, 1, 2])))
items = [a, b, c, d, e, f, g]
database = [trans1, trans2, trans3, trans4, trans5]

for i in items:
    i.twu = calculate_transaction_weight_utility({i}, database)
    print(i.item + ", " + str(i.twu))

A, 45
B, 56
C, 76
D, 48
E, 73
F, 25
G, 28


In [65]:
# a < b
def check_order_condition(a: Item, b: Item):
    if a.utility < 0 and b.utility > 0:
        return True
    elif a.utility * b.utility > 0:
        return a.twu > b.twu
    return False

def check_order_item_and_set(ik: Item, X: set[Item]) -> bool:
    for i in X:
        if i != ik:
            if check_order_condition(ik, i) == False:
                return False
    return True

In [66]:
def calculate_remaining_utility_of_item_set_in_trans(
    items: set[Item], trans: Transaction
):
    ru = 0
    trans_items: set[Item] = trans.items_quantities.keys()
    if items.issubset(trans_items):
        for item in trans_items:
            if item.utility > 0:
                if check_order_item_and_set(item, items) == True:
                    if item not in items:
                        print(item)
                        ru += item.utility * trans.items_quantities.get(item)
    return ru

F > G > D > B > E > C > A

In [67]:
print(trans4)
print(calculate_remaining_utility_of_item_set_in_trans({d, e}, trans4))

(tid = 4, frequencies = {(B,56): 4, (C,76): 3, (D,48): 3, (E,73): 1})
(C,76)
3
